In [2]:
from voice_agent_flow.llms import create_pydantic_azure_openai
import nest_asyncio
from pydantic_ai import Agent, AgentRunResultEvent, AgentStreamEvent
nest_asyncio.apply()
from pprint import pprint
import asyncio

model = create_pydantic_azure_openai(model_name = "gpt-4.1")

agent = Agent(model)




**Synchronous Run**

In [3]:
result_sync = agent.run_sync('What is the capital of Italy?')
print(result_sync.output)

The capital of Italy is **Rome**.


In [4]:
result_sync.all_messages()

[ModelRequest(parts=[UserPromptPart(content='What is the capital of Italy?', timestamp=datetime.datetime(2026, 2, 20, 16, 39, 0, 893507, tzinfo=datetime.timezone.utc))], run_id='b8452fec-53f8-4197-bb68-3b712a89ef10'),
 ModelResponse(parts=[TextPart(content='The capital of Italy is **Rome**.')], usage=RequestUsage(input_tokens=14, output_tokens=10, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}), model_name='gpt-4.1-2025-04-14', timestamp=datetime.datetime(2026, 2, 20, 16, 39, 2, tzinfo=TzInfo(UTC)), provider_name='openai', provider_url='https://azure-m7byy3cl-eastus2.cognitiveservices.azure.com/openai/', provider_details={'finish_reason': 'stop'}, provider_response_id='chatcmpl-DBNosNmRgfqm3dMTUnmXRWiqa0Khb', finish_reason='stop', run_id='b8452fec-53f8-4197-bb68-3b712a89ef10')]

**Asynchronous Run**

In [3]:
await agent.run('What is the capital of France?')

AgentRunResult(output='The capital of France is **Paris**.')

**Asyncronous Streaming Text**

In [5]:
async with agent.run_stream('What is the capital of the UK?') as response:
        async for text in response.stream_text(delta = True):
            print(text)

The capital of the United Kingdom (UK) is **London**.


**Asynchronous Streaming Events**

In [6]:
from pydantic import BaseModel

class Person(BaseModel):
    """Defines a person"""
    name:str
    aget:str
    
events: list[AgentStreamEvent | AgentRunResultEvent] = []

model = create_pydantic_azure_openai(model_name = "gpt-4.1")

agent = Agent(
    model,
    output_type= str | Person,
    instructions = "You are a helpful assistant that answers questions about anything, but when the user metions a person, you will reply with a json object defined by the schema."

)


async for event in agent.run_stream_events('My name is huan, 32 years old.'):
    events.append(event)
    print(event)



PartStartEvent(index=0, part=ToolCallPart(tool_name='final_result', args='', tool_call_id='call_P0v9wzkOs0MkKfYFe2Y2q8Xx'))
FinalResultEvent(tool_name='final_result', tool_call_id='call_P0v9wzkOs0MkKfYFe2Y2q8Xx')
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='{"', tool_call_id='call_P0v9wzkOs0MkKfYFe2Y2q8Xx'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='name', tool_call_id='call_P0v9wzkOs0MkKfYFe2Y2q8Xx'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='":"', tool_call_id='call_P0v9wzkOs0MkKfYFe2Y2q8Xx'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='hu', tool_call_id='call_P0v9wzkOs0MkKfYFe2Y2q8Xx'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='an', tool_call_id='call_P0v9wzkOs0MkKfYFe2Y2q8Xx'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='","', tool_call_id='call_P0v9wzkOs0MkKfYFe2Y2q8Xx'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='aget', tool_call_id='call_P0v9wzkOs0MkKfYFe2Y2q8Xx'

In [8]:
res = agent.run_sync('My name is huan, 32 years old.')

In [10]:
res.all_messages()

[ModelRequest(parts=[UserPromptPart(content='My name is huan, 32 years old.', timestamp=datetime.datetime(2026, 2, 20, 16, 42, 21, 604356, tzinfo=datetime.timezone.utc))], instructions='You are a helpful assistant that answers questions about anything, but when the user metions a person, you will reply with a json object defined by the schema.', run_id='4ac8791f-be7a-4ed4-a3cf-737b0c73f63c'),
 ModelResponse(parts=[ToolCallPart(tool_name='final_result', args='{"name":"huan","aget":"32"}', tool_call_id='call_ndaJB6yitsBbqbBYUwfbNWTr')], usage=RequestUsage(input_tokens=89, output_tokens=20, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}), model_name='gpt-4.1-2025-04-14', timestamp=datetime.datetime(2026, 2, 20, 16, 42, 22, tzinfo=TzInfo(UTC)), provider_name='openai', provider_url='https://azure-m7byy3cl-eastus2.cognitiveservices.azure.com/openai/', provider_details={'finish_reason': 'tool_calls'}, provider_response_id='

In [6]:
def weather_tool(location: str) -> str:
    """Search the weather for a location."""
    return f"The weather in {location} is sunny."

agent = Agent(model, tools = [weather_tool])

events = []
async for event in agent.run_stream_events('What is the weather in New York City?'):
    print(event)
    events.append(event)

PartStartEvent(index=0, part=ToolCallPart(tool_name='weather_tool', args='', tool_call_id='call_Qe2A3si22k0OuPVFFgCSMezw'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='{"', tool_call_id='call_Qe2A3si22k0OuPVFFgCSMezw'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='location', tool_call_id='call_Qe2A3si22k0OuPVFFgCSMezw'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='":"', tool_call_id='call_Qe2A3si22k0OuPVFFgCSMezw'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='New', tool_call_id='call_Qe2A3si22k0OuPVFFgCSMezw'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta=' York', tool_call_id='call_Qe2A3si22k0OuPVFFgCSMezw'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta=' City', tool_call_id='call_Qe2A3si22k0OuPVFFgCSMezw'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='"}', tool_call_id='call_Qe2A3si22k0OuPVFFgCSMezw'))
PartEndEvent(index=0, part=ToolCallPart(tool_name='weather_tool', args='{"loca

In [7]:
from pydantic_ai import ToolReturnPart
from pydantic_ai import ModelRequest
from pydantic_ai import ModelResponse
from pydantic_core import to_jsonable_python
from pydantic_ai import ModelMessagesTypeAdapter
from pydantic_ai import ToolCallPart

tool_return_part = ToolReturnPart(
    **{
        'tool_name': 'weather_tool',
        'content': 'The weather in New York City is sunny.',
        'tool_call_id': 'call_0tEPo1T8sE1GvoibSww0pqlD',
        'metadata': None,
        'timestamp': '2026-02-18T14:59:09.619076Z',
        'part_kind': 'tool-return'
    }
)

model_request_part = ModelRequest(parts = [tool_return_part])
model_request_part


ModelRequest(parts=[ToolReturnPart(tool_name='weather_tool', content='The weather in New York City is sunny.', tool_call_id='call_0tEPo1T8sE1GvoibSww0pqlD', timestamp='2026-02-18T14:59:09.619076Z')])

In [8]:
tool_call_part = ToolCallPart(
    **{
        'tool_name': 'weather_tool',
        'args': '{"location":"New York City"}',
        'tool_call_id': 'call_0tEPo1T8sE1GvoibSww0pqlD',
        'id': None,
        'provider_details': None,
        'part_kind': 'tool-call'
    }
)


model_response_part = ModelResponse(parts = [tool_call_part])
ModelMessagesTypeAdapter.validate_python([to_jsonable_python(model_response_part)])

[ModelResponse(parts=[ToolCallPart(tool_name='weather_tool', args='{"location":"New York City"}', tool_call_id='call_0tEPo1T8sE1GvoibSww0pqlD')], usage=RequestUsage(), timestamp=datetime.datetime(2026, 2, 20, 16, 30, 1, 791552, tzinfo=TzInfo(UTC)))]